In [179]:
import numpy as np
from datetime import datetime as dt
import pandas as pd, datetime as dt

In [116]:
cd C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data

C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data


In [202]:
user = pd.read_csv('user_processed.csv',sep=',')
listening = pd.read_csv('listening_processed.csv',sep=',')
order = pd.read_csv('order_processed.csv',sep=',')
clustering = pd.read_csv('clustering_processed.csv',sep=',')


In [203]:
# change column name
order = order.rename(columns = {"OrderID (PK)": "oderID", "Transaction Date" : "transaction_date","UserID (FK)" : "userID", "Bill Type": "bill_type", "Total Amount": "total_amount" })
clustering = clustering.rename(columns = {"User_ID (FK)": "userID", "Type_user": "type_user", "MainCluster_ID" : "main_cluseringID", "MainCluster_Description" : "main_clustering_description"})
listening = listening.rename(columns = {"PlaylistID (PK)": "playlistID", "Playlist Name":"playlist_name", "Category":"cat", "Sub Category" :"sub_cat", "Playlist Type" :"palylist_type","Playlist Duration (min)": "playlist_duration","Actual Duration (min)":"actual_duation","UserID (FK)":"userID", "Listening Time":"listening_time", "Listening Date" :"listening_date", "Listening Datetime":"listening_datetime"})
user = user.rename(columns = {"UserID (PK)":"userID", "VIP hay Free":"user_type", "Gender":"gender", "Year of Birth":"year_of_birth","Age Range": "age_range", "Age":"age", "Region": "region","Operation Systems":"operation_system","Registration Date":"registration_date", "Total Listerning":"total_listening", "Total Listening Time (Min)":"total_listening_time",   })

# change date time
# xu li datetime
order.transaction_date = pd.to_datetime(order['transaction_date'], format='%Y-%m-%d')
user.registration_date = pd.to_datetime(user['registration_date'], format='%Y-%m-%d')
listening.listening_datetime = pd.to_datetime(listening['listening_datetime'], format='%Y-%m-%d')

# change date by weekly
def adding_by_week(df, date_field):
    # df["by_week"] = df[[date_field]].copy()
    # df.resample('W-Mon', on=date_field).sum().reset_index().sort_values(by=date_field)
    # order1.resample('W-Mon', on='transaction_date').sum().reset_index().sort_values(by='transaction_date')
    df['by_week'] = df.apply(lambda row: row[date_field] - dt.timedelta(days=row[date_field].weekday()), axis=1)
    df = df['by_week'].groupby(df['by_week']).count()

adding_by_week(order, "transaction_date")
adding_by_week(user, "registration_date")
adding_by_week(listening, "listening_datetime")

In [204]:
## fillna for the user Table

user.operation_system = user.operation_system.fillna('none')
user["gender"] = user.gender.replace("no information","none")
user["gender"] = user.gender.replace("other","none")

## add some features related to time of the listening Table

listening['hour_listening'] = listening.listening_datetime.dt.hour
listening['dayofweek'] = listening['listening_datetime'].dt.dayofweek

listening["timepoint_of_the_day"] = listening.hour_listening.apply(lambda x : 
                                                             "late night" if x < 4 else
                                                             "early morning" if x < 7 else
                                                             "morning" if x < 11 else
                                                             "early afternoon" if x < 16 else
                                                             "afternoon" if x < 18 else
                                                             "evening"
                                                            )

listening['dayofweek'] =listening.dayofweek.apply(lambda x :  "Mon" if x == 0 else
                                                             "Tue" if x ==1 else
                                                             "Wed" if x ==2 else
                                                             "Thu" if x == 3 else
                                                             "Fri" if x == 4 else
                                                             "Sat" if x == 5 else
                                                             "Sun" 
                                                            )

listening['Workingday'] = listening["dayofweek"].apply(lambda x : 0 if x == 'Sun' or x == 'Sat' else 1)


In [210]:
user.head(3)

,userID,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3,2020-11-23
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6,2020-08-24
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5,2020-08-17


In [211]:
listening.head(3)

,playlistID,playlist_name,cat,sub_cat,palylist_type,playlist_duration,actual_duation,userID,listening_time,listening_date,listening_datetime,Unnamed: 11,by_week,hour_listening,dayofweek,timepoint_of_the_day,Workingday
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,06:05:00,03-12-2020,2020-12-03 06:05:00,NaN,2020-11-30 06:05:00,6,Thu,early morning,1
1,4577,Tư Duy Tích Cực - Bạn Là Những Gì Bạn Nghĩ (Nh...,Sách nói,Hạnh phúc,vip,79.8,15.9,56112,12:49:00,30-10-2020,2020-10-30 12:49:00,NaN,2020-10-26 12:49:00,12,Fri,early afternoon,1
2,4169,50 thiên thần của bạn: Giải phóng bản thân khỏ...,Sách nói,Tâm linh,vip,324.8,6.8,94362,00:17:00,15-12-2020,2020-12-15 00:17:00,NaN,2020-12-14 00:17:00,0,Tue,late night,1


### mục tiêu là mình làm xem nhóm nào đóng góp nhiều revenue nhất
1. paid user: dựa vào total amount
2. free user: dựa vào duration, số lượt nghe